# ACME OPTIMIZATION CASE STUDY

**Author**: Sruthi Keerthana Nuttakki

**Date**: 2/23/2025

---

## Notebook Overview

1. [Introduction](#section1)  
2. [Create Synthetic Dataset](#section2)  
3. [Define Optimization Function](#section3)  
4. [Solve Different Scenarios](#section4)  
5. [Visualize Results](#section5)  
6. [Extend to 5 Years](#section6)

---


**1. INTRODUCTION**

<a id="intro"></a>

This notebook implements the ProMazo/Acme case study requirements:

  1. Create a synthetic dataset replicating
Acme’s hierarchy (Portfolio → Geography → Category → Brand → Segment).
  2. Enforce Trend (min/max growth) and Contribution (min/max share) constraints for each segment.
  3. Provide four optimization scenarios:
      *   Maximize Sales
      *   Maximize Margin
      *   Hit a Sales Target While Maximizing Margin
      *   Hit a Sales Target While Maximizing Margin
  4.  Visualize the results:
  
       (a) Old vs. New bar

       (b) Distribution pie

       (c) Brand-level bar
  5. Show a 5-year iterative projection.



In [ ]:
# Importing necessary libraries
import warnings
warnings.filterwarnings("ignore", message=".*missing ScriptRunContext.*")
import warnings
warnings.simplefilter("ignore")


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display
from pulp import (
    LpProblem, LpMaximize, LpVariable,
    lpSum, LpStatus
)


plt.style.use('seaborn-v0_8')
plt.rcParams.update({
    "figure.autolayout": True,
    "font.size": 11
})

pio.renderers.default = "colab"
np.random.seed(42)

**3. SYNTHETIC DATA GENERATION**

We generate a DataFrame reflecting Acme's hierarchy with columns:
- Portfolio, Geography, Category, Brand, Segment
- Initial Sales, Margin
- Min Trend, Max Trend
- Min Contribution, Max Contribution

Each Brand’s segments must have total min_contribution ≤ 1 and total max_contribution ≥ 1 so the constraints remain feasible.

In [ ]:
def create_synthetic_data():
    portfolios = ["Skin/Body", "Fragrance + Color Cosmetics", "Hair/APDO"]
    geographies = ["North America", "Europe", "South America", "Asia"]
    categories = ["Fragrance", "Hair Dye", "Face Make-Up", "Make-Up Brushes", "Tools"]
    brands = ["Bobbi Brown", "Elizabeth Arden", "Aveda", "Kilian", "Frederic Malle", "Balmain"]
    segments = ["Lipstick", "Mascara", "Toner", "Bronzer", "Hair Dye", "Foundation", "BrushKit", "CombKit"]

    rows = []
    for seg in segments:
        b = np.random.choice(brands)
        row = {
            "Portfolio": np.random.choice(portfolios),
            "Geography": np.random.choice(geographies),
            "Category": np.random.choice(categories),
            "Brand": b,
            "Segment": seg,
            "Initial Sales": np.random.randint(1_000_000, 5_000_000),
            "Margin": round(np.random.uniform(0.10, 0.25), 3),
            "Min Trend": round(np.random.uniform(-0.03, 0.0), 3),
            "Max Trend": round(np.random.uniform(0.01, 0.06), 3),
            "Min Contribution": round(np.random.uniform(0.01, 0.05), 3),
            "Max Contribution": round(np.random.uniform(0.06, 0.20), 3)
        }
        rows.append(row)
    df = pd.DataFrame(rows)

    # Ensure brand-level min_contrib sum <=1, max_contrib sum >=1
    for br in df["Brand"].unique():
        idxs = df[df["Brand"]==br].index
        sum_min = df.loc[idxs,"Min Contribution"].sum()
        sum_max = df.loc[idxs,"Max Contribution"].sum()
        if sum_min>1.0:
            df.loc[idxs, "Min Contribution"] = df.loc[idxs,"Min Contribution"]/sum_min*0.9
        if sum_max<1.0:
            df.loc[idxs, "Max Contribution"] = df.loc[idxs,"Max Contribution"]/sum_max*1.1
    df.reset_index(drop=True, inplace=True)
    return df

synthetic_data = create_synthetic_data()
display(synthetic_data)

# Quick bar chart of initial sales
fig_init = px.bar(
    synthetic_data,
    x="Segment",
    y="Initial Sales",
    color="Brand",
    title="Initial Sales by Segment",
    width=700, height=400,
    template="simple_white",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig_init.update_layout(font=dict(size=10))
fig_init.show()

,Portfolio,Geography,Category,Brand,Segment,Initial Sales,Margin,Min Trend,Max Trend,Min Contribution,Max Contribution
0,Skin/Body,South America,Face Make-Up,Kilian,Lipstick,2692743,0.190,-0.025,0.018,0.012,0.608869
1,Hair/APDO,Europe,Tools,Kilian,Mascara,1787201,0.208,-0.002,0.010,0.050,0.491131
2,Fragrance + Color Cosmetics,North America,Make-Up Brushes,Elizabeth Arden,Toner,3583384,0.144,-0.012,0.017,0.022,0.444000
3,Fragrance + Color Cosmetics,South America,Face Make-Up,Balmain,Bronzer,3688875,0.177,-0.012,0.012,0.034,1.100000
4,Skin/Body,Asia,Hair Dye,Elizabeth Arden,Hair Dye,3795513,0.146,-0.027,0.044,0.028,0.308000
5,Hair/APDO,South America,Fragrance,Frederic Malle,Foundation,3557489,0.199,-0.021,0.036,0.032,1.100000
6,Fragrance + Color Cosmetics,Asia,Hair Dye,Elizabeth Arden,BrushKit,3658505,0.234,-0.012,0.056,0.014,0.348000
7,Fragrance + Color Cosmetics,Asia,Tools,Bobbi Brown,CombKit,1023247,0.212,-0.014,0.039,0.049,1.100000


In [ ]:
# ### Contribution Constraints Validation
brand_contrib = synthetic_data.groupby("Brand").agg(
    Min_Contrib=('Min Contribution', 'sum'),
    Max_Contrib=('Max Contribution', 'sum')
).reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=brand_contrib["Brand"],
    y=brand_contrib["Min_Contrib"],
    name='Min Contribution',
    marker_color='#406882'  # Muted navy
))
fig.add_trace(go.Bar(
    x=brand_contrib["Brand"],
    y=brand_contrib["Max_Contrib"],
    name='Max Contribution',
    marker_color='#7AA5D2'  # Soft blue
))

fig.update_layout(
    title="<b>Brand Contribution Constraints Validation</b>",
    barmode='group',
    width=600,  # Reduced width
    height=400,  # Reduced height
    template="plotly_white",
    yaxis_title="Contribution Ratio",
    margin=dict(l=50, r=50, t=80, b=50)
)

fig.show()

**3. OPTIMIZATION ENGINE**

We define a base model with:
- new_sales_s[seg] for each segment
- Trend constraints
- brand_total variables to sum each brand's new sales
- min_contribution <= new_sales_seg / brand_total <= max_contribution

Objective is set to an empty sum; we define the scenario objectives separately.


In [ ]:
def build_base_model(df):
    model = LpProblem("Acme_BaseModel", sense=LpMaximize)
    seg_list = df["Segment"].tolist()
    new_sales_s = {s: LpVariable(f"new_sales_{s}", lowBound=0) for s in seg_list}

    # Trend
    for _, row in df.iterrows():
        seg = row["Segment"]
        init_s = row["Initial Sales"]
        min_t = row["Min Trend"]
        max_t = row["Max Trend"]
        model += (new_sales_s[seg] >= init_s*(1+min_t))
        model += (new_sales_s[seg] <= init_s*(1+max_t))

    # Brand totals
    brands = df["Brand"].unique()
    brand_tot = {b: LpVariable(f"brand_total_{b}", lowBound=0) for b in brands}
    for b in brands:
        segs_b = df.loc[df["Brand"] == b, "Segment"].tolist()
        model += (brand_tot[b] == lpSum([new_sales_s[s] for s in segs_b]))

    # Contribution
    for _, row in df.iterrows():
        seg = row["Segment"]
        b   = row["Brand"]
        min_c = row["Min Contribution"]
        max_c = row["Max Contribution"]
        model += (new_sales_s[seg] >= min_c * brand_tot[b])
        model += (new_sales_s[seg] <= max_c * brand_tot[b])

    # Dummy objective
    model.setObjective(lpSum([]))
    return model, new_sales_s

base_model, new_sales_dict = build_base_model(synthetic_data)
print("Number of constraints in base model:", len(base_model.constraints))

Number of constraints in base model: 37


**4. STRATERGIC SCENARIO ANALYSIS**

- 'max_sales': objective = sum of new_sales
- 'max_margin': objective = sum of new_sales * margin
- 'sales_target': sum(new_sales) >= target, objective => sum(new_sales * margin)
- 'margin_target': sum(new_sales * margin) >= target, objective => sum(new_sales)

We build a new problem each time from the base constraints, set the objective, then solve.



In [ ]:
# 4) SCENARIOS & SOLVER
# =============================
def solve_scenario(df, base_model, new_sales_s, scenario, target_value=None):
    scenario_model = LpProblem("AcmeScenario", sense=LpMaximize)
    scenario_model += base_model.objective
    for c in base_model.constraints.values():
        scenario_model.addConstraint(c)

    seg_list = df["Segment"].tolist()
    if scenario=="max_sales":
        obj = lpSum([new_sales_s[s] for s in seg_list])
    elif scenario=="max_margin":
        obj = lpSum([new_sales_s[r["Segment"]]*r["Margin"] for _, r in df.iterrows()])
    elif scenario=="sales_target":
        scenario_model += lpSum([new_sales_s[s] for s in seg_list]) >= target_value
        obj = lpSum([new_sales_s[r["Segment"]]*r["Margin"] for _, r in df.iterrows()])
    elif scenario=="margin_target":
        scenario_model += lpSum([new_sales_s[r["Segment"]]*r["Margin"] for _, r in df.iterrows()]) >= target_value
        obj = lpSum([new_sales_s[s] for s in seg_list])
    else:
        raise ValueError("Invalid scenario")

    scenario_model.setObjective(obj)
    scenario_model.solve()
    status = LpStatus[scenario_model.status]

    # Build solution DataFrame
    results = []
    for _, row in df.iterrows():
        seg = row["Segment"]
        old_s = row["Initial Sales"]
        val = new_sales_s[seg].varValue
        margin_val = row["Margin"]
        growth_rate = (val - old_s)/old_s if old_s!=0 else 0
        profit_val = val*margin_val
        results.append({
            "Segment": seg,
            "Brand": row["Brand"],
            "Old Sales": old_s,
            "New Sales": val,
            "Growth Rate": round(growth_rate,3),
            "Margin": margin_val,
            "Profit": round(profit_val,2)
        })
    sol_df = pd.DataFrame(results)
    return sol_df, status

**5) VISUALIZATIONS**

We produce 3 Plotly charts for each scenario solution:

1) Old vs New bar

2) Distribution pie

3) Brand-level bar

In [ ]:
# 5) VISUALIZATION
# =============================
def visualize_solution(sol_df, scenario_name):
    # (1) Old vs New bar
    fig_bar = px.bar(
        sol_df,
        x="Segment",
        y=["Old Sales","New Sales"],
        barmode="group",
        title=f"Old vs New Sales ({scenario_name})",
        template="simple_white",
        width=700,
        height=400,
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    fig_bar.update_layout(font=dict(size=10))
    fig_bar.show(renderer="colab")

    # (2) Pie of new sales
    fig_pie = px.pie(
        sol_df,
        names="Segment",
        values="New Sales",
        title=f"Distribution of New Sales ({scenario_name})",
        template="simple_white",
        width=700,
        height=400,
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    fig_pie.update_layout(font=dict(size=10))
    fig_pie.show(renderer="colab")

    # (3) Brand-level sums
    brand_sums = sol_df.groupby("Brand")["New Sales"].sum().reset_index()
    fig_brand = px.bar(
        brand_sums,
        x="Brand",
        y="New Sales",
        title=f"Brand-Level Total New Sales ({scenario_name})",
        template="simple_white",
        width=700,
        height=400,
        color_discrete_sequence=["#2ca02c"]
    )
    fig_brand.update_layout(font=dict(size=10))
    fig_brand.show(renderer="colab")



In [ ]:
print("\nRunning 4 scenarios.")

scenario_list = [
    ("max_sales", None),
    ("max_margin", None),
    ("sales_target", 20_000_000),
    ("margin_target", 3_000_000)
]

for (scen, t_val) in scenario_list:
    print(f"\n=== Scenario: {scen}, target={t_val if t_val else 'N/A'} ===")
    sol_df, stat = solve_scenario(synthetic_data, base_model, new_sales_dict, scen, t_val)
    print("Status:", stat)

    # ALWAYS display the DataFrame + plots, even if infeasible
    display(sol_df)

    # Summaries
    total_sales = sol_df["New Sales"].sum()
    total_profit = sol_df["Profit"].sum()
    print(f"Total Sales: {total_sales:,.0f}, Profit: {total_profit:,.0f}")

    # Plots
    visualize_solution(sol_df, scen)



Running 4 scenarios.

=== Scenario: max_sales, target=N/A ===
Status: Infeasible


,Segment,Brand,Old Sales,New Sales,Growth Rate,Margin,Profit
0,Lipstick,Kilian,2692743,2741212.4,0.018,0.190,520830.36
1,Mascara,Kilian,1787201,1805073.0,0.010,0.208,375455.18
2,Toner,Elizabeth Arden,3583384,3644301.5,0.017,0.144,524779.42
3,Bronzer,Balmain,3688875,3733141.5,0.012,0.177,660766.05
4,Hair Dye,Elizabeth Arden,3795513,3341569.8,-0.120,0.146,487869.19
5,Foundation,Frederic Malle,3557489,3685558.6,0.036,0.199,733426.16
6,BrushKit,Elizabeth Arden,3658505,3863381.3,0.056,0.234,904031.22
7,CombKit,Bobbi Brown,1023247,1063153.6,0.039,0.212,225388.56


Total Sales: 23,877,392, Profit: 4,432,546



=== Scenario: max_margin, target=N/A ===
Status: Infeasible


,Segment,Brand,Old Sales,New Sales,Growth Rate,Margin,Profit
0,Lipstick,Kilian,2692743,2741212.4,0.018,0.190,520830.36
1,Mascara,Kilian,1787201,1805073.0,0.010,0.208,375455.18
2,Toner,Elizabeth Arden,3583384,3644301.5,0.017,0.144,524779.42
3,Bronzer,Balmain,3688875,3733141.5,0.012,0.177,660766.05
4,Hair Dye,Elizabeth Arden,3795513,3341569.8,-0.120,0.146,487869.19
5,Foundation,Frederic Malle,3557489,3685558.6,0.036,0.199,733426.16
6,BrushKit,Elizabeth Arden,3658505,3863381.3,0.056,0.234,904031.22
7,CombKit,Bobbi Brown,1023247,1063153.6,0.039,0.212,225388.56


...

,Segment,Brand,Old Sales,New Sales,Growth Rate,Margin,Profit
0,Lipstick,Kilian,2692743,2741212.4,0.018,0.190,520830.36
1,Mascara,Kilian,1787201,1805073.0,0.010,0.208,375455.18
2,Toner,Elizabeth Arden,3583384,3644301.5,0.017,0.144,524779.42
3,Bronzer,Balmain,3688875,3733141.5,0.012,0.177,660766.05
4,Hair Dye,Elizabeth Arden,3795513,3341569.8,-0.120,0.146,487869.19
5,Foundation,Frederic Malle,3557489,3685558.6,0.036,0.199,733426.16
6,BrushKit,Elizabeth Arden,3658505,3863381.3,0.056,0.234,904031.22
7,CombKit,Bobbi Brown,1023247,1063153.6,0.039,0.212,225388.56


Total Sales: 23,877,392, Profit: 4,432,546



=== Scenario: margin_target, target=3000000 ===
Status: Infeasible


,Segment,Brand,Old Sales,New Sales,Growth Rate,Margin,Profit
0,Lipstick,Kilian,2692743,2741212.4,0.018,0.190,520830.36
1,Mascara,Kilian,1787201,1805073.0,0.010,0.208,375455.18
2,Toner,Elizabeth Arden,3583384,3644301.5,0.017,0.144,524779.42
3,Bronzer,Balmain,3688875,3733141.5,0.012,0.177,660766.05
4,Hair Dye,Elizabeth Arden,3795513,3341569.8,-0.120,0.146,487869.19
5,Foundation,Frederic Malle,3557489,3685558.6,0.036,0.199,733426.16
6,BrushKit,Elizabeth Arden,3658505,3863381.3,0.056,0.234,904031.22
7,CombKit,Bobbi Brown,1023247,1063153.6,0.039,0.212,225388.56


Total Sales: 23,877,392, Profit: 4,432,546


**6. LONG-TERM STRATERGIC PLANNING (5-Year Financial Projection System)**

We run 'max_sales' each year for 5 years,updating 'Initial Sales' with the new solution each iteration.

Then produce a brand-level line chart summarizing all 5 years.

In [ ]:
# 7) FIVE-YEAR PROJECTION
# =============================
def run_five_year_projection(df):
    results_dict = {}
    working_df = df.copy()
    for y in range(1,6):
        print(f"\n=== YEAR {y} ===")
        mod_y, ns_y = build_base_model(working_df)
        sol_y, stt = solve_scenario(working_df, mod_y, ns_y, "max_sales")
        print("Solver Status:", stt)
        ssum = sol_y["New Sales"].sum()
        psum = sol_y["Profit"].sum()
        print(f"Year {y} => Sales: {ssum:,.0f}, Profit: {psum:,.0f}")
        results_dict[y] = sol_y
        # Next year
        working_df["Initial Sales"] = sol_y["New Sales"].values
    return results_dict

def visualize_five_year_projection(year_dict):
    # Summarize brand-level new sales across years, smaller fig size
    gather_rows = []
    for year_n, df_sol in year_dict.items():
        brand_grp = df_sol.groupby("Brand")["New Sales"].sum().reset_index()
        for _, row_b in brand_grp.iterrows():
            gather_rows.append({
                "Year": year_n,
                "Brand": row_b["Brand"],
                "Sales": row_b["New Sales"]
            })
    big_df = pd.DataFrame(gather_rows)
    fig_line = px.line(
        big_df,
        x="Year",
        y="Sales",
        color="Brand",
        markers=True,
        title="Brand Sales Over 5 Years (Max Sales)",
        template="simple_white",
        width=700, height=400,
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    fig_line.update_layout(font=dict(size=10))
    fig_line.show(renderer="colab")

print("\nRunning a 5-Year Projection...\n")
data_multi = create_synthetic_data()
five_year_solutions = run_five_year_projection(data_multi)
visualize_five_year_projection(five_year_solutions)





Running a 5-Year Projection...


=== YEAR 1 ===
Solver Status: Infeasible
Year 1 => Sales: 18,573,109, Profit: 3,064,048

=== YEAR 2 ===
Solver Status: Infeasible
Year 2 => Sales: 15,975,700, Profit: 2,670,642

=== YEAR 3 ===
Solver Status: Infeasible
Year 3 => Sales: 15,744,421, Profit: 2,587,000

=== YEAR 4 ===
Solver Status: Optimal
Year 4 => Sales: 16,259,398, Profit: 2,667,030

=== YEAR 5 ===
Solver Status: Optimal
Year 5 => Sales: 16,804,763, Profit: 2,752,329


In [ ]:
# acme_dashboard.py - Enhanced Streamlit version
import streamlit as st
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings

# Custom CSS styling
st.markdown("""
<style>
    .main {
        background-color: #F5F7FA;
        padding: 2rem;
    }
    h1 {
        color: #2A3F5F;
        border-bottom: 2px solid #2A3F5F;
        padding-bottom: 0.5rem;
    }
    .stButton>button {
        background-color: #4A90E2;
        color: white;
        border-radius: 5px;
        padding: 0.5rem 1rem;
    }
    .stSelectbox, .stNumberInput {
        background-color: white;
        border-radius: 4px;
        padding: 0.5rem;
    }
    .dataframe {
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        border-radius: 8px;
    }
    .status-box {
        padding: 1rem;
        border-radius: 5px;
        margin: 1rem 0;
    }
</style>
""", unsafe_allow_html=True)

st.markdown("# 🚀 ACME Strategic Optimization Dashboard")
st.markdown("### Data-Driven Insights for Corporate Portfolio Optimization")
st.markdown("""
Welcome to the **ACME Strategic Optimization Dashboard**.
This platform implements the ProMazo/Acme case study requirements by:
- Generating a synthetic dataset that replicates Acme’s hierarchy (Portfolio → Geography → Category → Brand → Segment),
- Enforcing Trend and Contribution constraints,
- Solving four optimization scenarios,
- Visualizing results with interactive charts,
- And projecting 5-year performance.
""")

# Generate or upload synthetic data
with st.sidebar:
    st.header("Configuration")
    if st.button("✨ Generate Synthetic Data", use_container_width=True):
        df = create_synthetic_data()
        st.session_state["data"] = df

if "data" in st.session_state:
    df = st.session_state["data"]

    # Scenario selection
    with st.expander("⚙️ Optimization Settings", expanded=True):
        col1, col2 = st.columns(2)
        with col1:
            scenario_option = st.selectbox(
                "Select Strategy",
                ["max_sales", "max_margin", "sales_target", "margin_target"],
                format_func=lambda x: x.replace("_", " ").title()
            )
        with col2:
            target = None
            if scenario_option in ["sales_target", "margin_target"]:
                target = st.number_input(
                    "🎯 Target Value",
                    value=20_000_000,
                    format="%d",
                    help="Enter required target value for the scenario"
                )

    if st.button("📈 Run Optimization", use_container_width=True):
        with st.spinner("Optimizing..."):
            base_mod, new_sales_vars = build_base_model(df)
            sol_df, status = solve_scenario(df, base_mod, new_sales_vars, scenario_option, target)

            # Status indicator
            status_color = "#4CAF50" if status == "Optimal" else "#F44336"
            st.markdown(f"""
            <div class="status-box" style="background-color: {status_color}20; border: 1px solid {status_color};">
                <h4 style="color: {status_color}; margin:0;">Solver Status: {status}</h4>
            </div>
            """, unsafe_allow_html=True)

            # Formatted results
            st.subheader("📊 Optimization Results")
            formatted_df = sol_df.style.format({
                "Old Sales": "${:,.0f}",
                "New Sales": "${:,.0f}",
                "Growth Rate": "{:.1%}",
                "Margin": "{:.1%}",
                "Profit": "${:,.0f}"
            })
            st.dataframe(formatted_df, use_container_width=True)

            # Key metrics
            col1, col2 = st.columns(2)
            with col1:
                st.metric("Total Sales", f"${sol_df['New Sales'].sum():,.0f}")
            with col2:
                st.metric("Total Profit", f"${sol_df['Profit'].sum():,.0f}")

            # Visualizations
            st.subheader("📈 Visual Analysis")

            # Create tabs for charts
            tab1, tab2, tab3 = st.tabs(["Sales Comparison", "Sales Distribution", "Brand Performance"])

            with tab1:
                fig_bar = px.bar(
                    sol_df, x="Segment", y=["Old Sales", "New Sales"],
                    barmode="group", title=f"Old vs New Sales ({scenario_option.replace('_', ' ').title()})",
                    template="plotly_white", height=400,
                    color_discrete_sequence=["#4A90E2", "#50C878"]
                )
                st.plotly_chart(fig_bar, use_container_width=True)

            with tab2:
                fig_pie = px.pie(
                    sol_df, names="Segment", values="New Sales",
                    title=f"Sales Distribution ({scenario_option.replace('_', ' ').title()})",
                    template="plotly_white", height=500,
                    color_discrete_sequence=px.colors.sequential.Blues_r
                )
                st.plotly_chart(fig_pie, use_container_width=True)

            with tab3:
                brand_sums = sol_df.groupby("Brand")["New Sales"].sum().reset_index()
                fig_brand = px.bar(
                    brand_sums, x="Brand", y="New Sales",
                    title=f"Brand Performance ({scenario_option.replace('_', ' ').title()})",
                    template="plotly_white", height=400,
                    color="Brand", color_discrete_sequence=px.colors.qualitative.Pastel
                )
                st.plotly_chart(fig_brand, use_container_width=True)

# Add footer
st.markdown("---")
st.markdown("📌 *ACME Corporation - Strategic Optimization Dashboard v2.0*")

2025-02-25 02:15:49.801 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:15:49.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:15:49.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:15:49.815 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:15:49.819 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:15:49.822 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:15:49.826 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 02:15:49.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()